Hello moonshine
<!-- https://mp.weixin.qq.com/s/NUjJir93A6RQbEltUR6l-w -->

In [ ]:
# from IPython.display import clear_output
# %pip install moonshine
# %pip install silero_vad onnxruntime sounddevice tokenizers einops
# %pip install onnxruntime-gpu

In [ ]:
# !git clone https://github.com/usefulsensors/moonshine.git

# D:\\coding\\hello-asr\\onnx\\base
# !huggingface-cli download UsefulSensors/moonshine  --local-dir . --local-dir-use-symlinks False
# clear_output()

使用 onnxruntime 包来运行 Moonshine 模型，不依赖torch

In [1]:
import wave
import numpy as np
import tokenizers

In [2]:
from onnx_model import MoonshineOnnxModel

In [3]:
model = MoonshineOnnxModel(models_dir="D:\\coding\\hello-asr\\onnx\\base")

def moonshine_infer(wav_file):
    with wave.open(wav_file) as f:
        params = f.getparams()
        assert (
            params.nchannels == 1
            and params.framerate == 16_000
            and params.sampwidth == 2
        ), f"wave file should have 1 channel, 16KHz, and int16"
        audio = f.readframes(params.nframes)
    audio = np.frombuffer(audio, np.int16) / 32768.0
    audio = audio.astype(np.float32)[None, ...]
    tokens = model.generate(audio)
    tokenizer = tokenizers.Tokenizer.from_file(
        "D:\\coding\\hello-asr\\moonshine\\moonshine\\assets\\tokenizer.json")
    text = tokenizer.decode_batch(tokens)
    return text

In [4]:
wav_file = "D:\\coding\\hello-asr\\moonshine\\moonshine\\assets\\beckett.wav"
moonshine_infer(wav_file)

['Ever tried ever failed, no matter try again fail again fail better.']

使用huggingface框架加载Moonshine模型进行asr语音识别

In [6]:
from transformers import AutoModelForSpeechSeq2Seq, PreTrainedTokenizerFast
import torchaudio
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    'usefulsensors/moonshine-base', trust_remote_code=True).to(device)
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    'usefulsensors/moonshine-base')

wav_file = "D:\\coding\\hello-asr\\moonshine\\moonshine\\assets\\beckett.wav"
audio, sr = torchaudio.load(wav_file)
if sr != 16000:
    audio = torchaudio.functional.resample(audio, sr, 16000)
audio = audio.to(device)
tokens = model(audio)
print(tokenizer.decode(tokens[0], skip_special_tokens=True))

Ever tried ever failed, no matter try again fail again fail better.
